# Data analysis for football matches

##### Data source:

In [24]:
import pandas as pd

### 1) Cleaning data

In [25]:
# First we have to read the data

# This CSV have all the matches
df_historical_data = pd.read_csv('data/fifa_worldcup_matches.csv')
# This have information about the matches schedule
df_fixture = pd.read_csv('data/fifa_worldcup_fixture.csv')
# This file have all the missing data
df_missing_data = pd.read_csv('data/fifa_worldcup_missing_data.csv')


#### 1.1) Cleaning df_fixture

Exploring the dataset, it can be found that there are data to correct in the "home" and "away" columns. For instance, in some cases, there are blank spaces within the text strings. This prevents filtering data when using a specific string because having a space makes the strings unequal.


In [26]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

#### 1.1) Cleaning and unifying df_historical_data and df_missing_data.

In [27]:
# First, I check if I see missing data
df_missing_data

# There doesn't appear to be any missing data, I can try using methods to confirm.
df_missing_data[df_missing_data['home'].isnull()]

# I can see that for the 'home' column, there are many rows with NaN values. In total, there are 64 rows with missing data.

,home,score,away,year
396,NaN,NaN,NaN,2010
397,NaN,NaN,NaN,2010
398,NaN,NaN,NaN,2010
399,NaN,NaN,NaN,2010
400,NaN,NaN,NaN,2010
...,...,...,...,...
455,NaN,NaN,NaN,2010
456,NaN,NaN,NaN,2010
457,NaN,NaN,NaN,2010
458,NaN,NaN,NaN,2010


In [28]:
# To remove these rows with missing data: 
df_missing_data.dropna(inplace=True)

Using dropna() removes missing data. By adding inplace=True, I'm overwriting my data to save it after removing the NaNs. Now I can check it.

In [29]:
df_missing_data[df_missing_data['home'].isnull()]


,home,score,away,year


Now, to combine the data, including the missing data, I can use the concat method. Setting 'ignore_index=True' allows the dataframes to be concatenated without considering their individual indices, which often cannot be related.

In [30]:
df_historical_data = pd.concat([df_historical_data,df_missing_data], ignore_index=True)

Now, since the missing_data file could contain information already present in the other dataframe, we need to remove duplicates. Additionally, after that, I will sort it by year.

In [31]:
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year', inplace=True)
df_historical_data

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
419,Brazil,2–0,Costa Rica,2018
420,Serbia,1–2,Switzerland,2018
421,Serbia,0–2,Brazil,2018
408,France,1–0,Peru,2018


#### 1.3) Cleaning df_historical_data

Exploring the data, there are matches that were not played. In some cases, "walkover" is indicated, but in other cases, an arbitrary value of 3-0 is used. This is not ideal if I wanted to predict match winners based on the number of goals, for example. So, knowing which matches have this result, I will search for them and remove them.

With this, I can see the matches where Sweden and Austria played, which is the walkover match we are looking for. To remove this element, I can obtain its index and delete it by index. Now, to remove it.

In [32]:
df_historical_data[df_historical_data['home'].str.contains('Sweden')&
                   df_historical_data['away'].str.contains('Austria')]

index_drop = df_historical_data[df_historical_data['home'].str.contains('Sweden')&
                   df_historical_data['away'].str.contains('Austria')].index

df_historical_data.drop(index=index_drop, inplace=True)

In the score column, while in most cases, we can see the results in numerical format, there are some that are in string format. To find those that do not contain numbers, we can use regular expressions.
[^] Finds characters not inside brackets.

In [33]:
df_historical_data[df_historical_data['score'].str.contains('[^\d–]')]

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Paula\AppData\Local\Temp\ipykernel_34972\3367592408.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_historical_data[df_historical_data['score'].str.contains('[^\d–]')]


,home,score,away,year
34,Italy,2–1 (a.e.t.),Czechoslovakia,1934
27,Italy,1–1 (a.e.t.),Spain,1934
24,Austria,3–2 (a.e.t.),France,1934
48,Brazil,1–1 (a.e.t.),Czechoslovakia,1938
42,Czechoslovakia,3–0 (a.e.t.),Netherlands,1938
...,...,...,...,...
443,Spain,1–1 (a.e.t.),Russia,2018
444,Croatia,1–1 (a.e.t.),Denmark,2018
448,Colombia,1–1 (a.e.t.),England,2018
452,Russia,2–2 (a.e.t.),Croatia,2018



For example, in those cases, we want to remove what is between parentheses. We can do this by replacing it with regular expressions, indicating that we want to remove everything that is not a digit or a hyphen.

In [34]:
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Paula\AppData\Local\Temp\ipykernel_34972\4242020390.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)


In [35]:
df_historical_data['home'] = df_historical_data['home'].str.strip()
df_historical_data['away'] = df_historical_data['away'].str.strip()

To make it easier to work with later on, I can split the score into two columns. The expand method allows me to do this in two columns instead of lists.

In [36]:
df_historical_data[['HomeGoals','AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data

,home,score,away,year,HomeGoals,AwayGoals
0,France,4–1,Mexico,1930,4,1
17,Uruguay,4–2,Argentina,1930,4,2
16,Uruguay,6–1,Yugoslavia,1930,6,1
15,Argentina,6–1,United States,1930,6,1
14,Paraguay,1–0,Belgium,1930,1,0
...,...,...,...,...,...,...
419,Brazil,2–0,Costa Rica,2018,2,0
420,Serbia,1–2,Switzerland,2018,1,2
421,Serbia,0–2,Brazil,2018,0,2
408,France,1–0,Peru,2018,1,0


Now, we need to remove the duplicated score column. Using axis=1 indicates that we want to remove a column.

In [38]:
df_historical_data.drop('score', axis=1, inplace=True)

# Rename some columns
df_historical_data.rename(columns={'home':'HomeTeam', 'away': 'AwayTeam','year': 'Year'}, inplace=True)
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
17,Uruguay,Argentina,1930,4,2
16,Uruguay,Yugoslavia,1930,6,1
15,Argentina,United States,1930,6,1
14,Paraguay,Belgium,1930,1,0
...,...,...,...,...,...
419,Brazil,Costa Rica,2018,2,0
420,Serbia,Switzerland,2018,1,2
421,Serbia,Brazil,2018,0,2
408,France,Peru,2018,1,0


Check the datatypes

In [43]:
df_historical_data.dtypes
df_historical_data = df_historical_data.astype({'HomeGoals':int, 'AwayGoals': int})
df_historical_data.dtypes

HomeTeam     object
AwayTeam     object
Year          int64
HomeGoals     int32
AwayGoals     int32
dtype: object

In [45]:
# Creo una nueva columna con goles totales
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
419,Brazil,Costa Rica,2018,2,0,2
420,Serbia,Switzerland,2018,1,2,3
421,Serbia,Brazil,2018,0,2,2
408,France,Peru,2018,1,0,1


### 2) Export clean data

In [48]:
df_historical_data.to_csv('data/clean_fifa_worldcup_matches.csv', index=False)
df_fixture.to_csv('data/clean_fifa_worldcup_fixture.csv', index=False)